In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn

In [2]:
import geopandas as gpd
import pandas as pd

## Load daily temperature 

In [3]:
years = [year for year in range(1960, 2020)]

In [4]:
data_temp = []
for year in years:
    temp_year = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/PRISM_Data/PRISM_daily_county_level/prism_daily_county_level_' + str(year) + '.csv', dtype = {'year': int, 'fips': str})
    temp_year = temp_year.loc[:, ~temp_year.columns.str.contains('^Unnamed')]
    data_temp.append(temp_year)

In [5]:
temp = pd.concat(data_temp)
temp.head()

,fips,date,tMean,tMin,tMax,prec,year
0,01001,1960-01-01,5.417989,2.635663,8.200316,2.279021,1960
1,01001,1960-01-02,7.005736,4.201073,9.810398,10.412733,1960
2,01001,1960-01-03,9.061477,5.235675,12.887278,16.011329,1960
3,01001,1960-01-04,5.334175,-0.744570,11.412921,0.216112,1960
4,01001,1960-01-05,6.495222,3.006032,9.984412,0.274311,1960


In [6]:
import datetime

In [7]:
city = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/climate_heatwave/50_city_county_level.csv', dtype = {'fips': str})
city.head()

,Unnamed: 0,NAME,CLASS,ST,STFIPS,PLACEFIPS,POP2010,STPLFIPS,geometry,fips
0,0,Seattle,city,WA,53,63000,608660,5363000,MULTIPOLYGON (((-13619425.9562 6033333.6701000...,53033
1,1,Portland,city,OR,41,59000,583776,4159000,MULTIPOLYGON (((-13646153.5976 5694643.1055999...,41051
2,2,San Francisco,city,CA,6,67000,805235,667000,MULTIPOLYGON (((-13623645.882399999 4546557.17...,06075
3,3,Fresno,city,CA,6,27000,494665,627000,"MULTIPOLYGON (((-13345406.2071 4398263.0374, -...",06019
4,4,Los Angeles,city,CA,6,44000,3792621,644000,MULTIPOLYGON (((-13162830.3816 3996851.7908999...,06037


In [8]:
temp = city[['NAME', 'fips']].merge(temp, on = 'fips', how = 'left')
temp.head()

,NAME,fips,date,tMean,tMin,tMax,prec,year
0,Seattle,53033,1960-01-01,-2.959183,-6.680200,0.761834,0.439367,1960
1,Seattle,53033,1960-01-02,-3.495462,-5.916871,-1.074053,0.533370,1960
2,Seattle,53033,1960-01-03,-3.105760,-6.547404,0.335883,0.613255,1960
3,Seattle,53033,1960-01-04,-2.948310,-5.962373,0.065753,0.443433,1960
4,Seattle,53033,1960-01-05,-0.580383,-2.889711,1.728945,1.753494,1960


In [9]:
def convertTime(time):
    return datetime.strptime(time, '%Y-%m-%d')

In [10]:
def getMonth(date):
    return(date.strftime('%b'))

In [11]:
temp['date'] = pd.to_datetime(temp['date'])
temp['month'] = temp['date'].apply(getMonth)

In [12]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}

In [13]:
temp['month'] = temp['month'].replace(month_replace)

In [14]:
import math

def water_vapor_pressure(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    return e


def apparent_temperature(temperature, water_vapor_pressure):
    A = -1.3 + 0.92 * temperature + 2.2 * water_vapor_pressure
    return A


temperature = 28 
water_vapor_pressure = water_vapor_pressure(temperature) 
apparent_temp = apparent_temperature(temperature, water_vapor_pressure)
print(f"Apparent temperature at {temperature} degrees Celsius and {water_vapor_pressure:.2f} kPa water vapor pressure is {apparent_temp:.2f} °C")

Apparent temperature at 28 degrees Celsius and 3.77 kPa water vapor pressure is 32.76 °C


In [15]:
def findApparentTemp(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    A = -1.3 + 0.92 * temperature + 2.2 * e
    return A

In [16]:
temp = temp.rename(columns = {'tMean': 'mean_temp', 'tMin': 'min_temp', 'tMax': 'max_temp'})

In [17]:
climate = temp[['year', 'month', 'date', 'fips', 'NAME', 'mean_temp', 'min_temp', 'max_temp']]
climate = climate.rename(columns = {'NAME': 'city_name'})
climate.head()

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp
0,1960,1,1960-01-01,53033,Seattle,-2.959183,-6.680200,0.761834
1,1960,1,1960-01-02,53033,Seattle,-3.495462,-5.916871,-1.074053
2,1960,1,1960-01-03,53033,Seattle,-3.105760,-6.547404,0.335883
3,1960,1,1960-01-04,53033,Seattle,-2.948310,-5.962373,0.065753
4,1960,1,1960-01-05,53033,Seattle,-0.580383,-2.889711,1.728945


In [18]:
climate['AT_mean'] = climate['mean_temp'].apply(findApparentTemp)
climate['AT_min'] = climate['min_temp'].apply(findApparentTemp)
climate['AT_max'] = climate['max_temp'].apply(findApparentTemp)
climate.head()

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1960,1,1960-01-01,53033,Seattle,-2.959183,-6.680200,0.761834,-2.940829,-6.629042,0.821056
1,1960,1,1960-01-02,53033,Seattle,-3.495462,-5.916871,-1.074053,-3.476559,-5.877712,-1.045205
2,1960,1,1960-01-03,53033,Seattle,-3.105760,-6.547404,0.335883,-3.087406,-6.498515,0.386174
3,1960,1,1960-01-04,53033,Seattle,-2.948310,-5.962373,0.065753,-2.929952,-5.922571,0.110983
4,1960,1,1960-01-05,53033,Seattle,-0.580383,-2.889711,1.728945,-0.545411,-2.871318,1.812890


### After data imputation, 97% of the counties have daily apparent tempeature data from 1960 to 2020

In [19]:
city_numbers = []
for year in years:
    dict_year = {}
    df_year = climate[climate['year'] == year]
    df_new = df_year.dropna(subset = 'AT_mean')
    numbers = len(df_new['city_name'].unique())
    dict_year[year] = numbers
    city_numbers.append(dict_year)

In [20]:
city_numbers

[{1960: 50},
 {1961: 50},
 {1962: 50},
 {1963: 50},
 {1964: 50},
 {1965: 50},
 {1966: 50},
 {1967: 50},
 {1968: 50},
 {1969: 50},
 {1970: 50},
 {1971: 50},
 {1972: 50},
 {1973: 50},
 {1974: 50},
 {1975: 50},
 {1976: 50},
 {1977: 50},
 {1978: 50},
 {1979: 50},
 {1980: 50},
 {1981: 50},
 {1982: 50},
 {1983: 50},
 {1984: 50},
 {1985: 50},
 {1986: 50},
 {1987: 50},
 {1988: 50},
 {1989: 50},
 {1990: 50},
 {1991: 50},
 {1992: 50},
 {1993: 50},
 {1994: 50},
 {1995: 50},
 {1996: 50},
 {1997: 50},
 {1998: 50},
 {1999: 50},
 {2000: 50},
 {2001: 50},
 {2002: 50},
 {2003: 50},
 {2004: 50},
 {2005: 50},
 {2006: 50},
 {2007: 50},
 {2008: 50},
 {2009: 50},
 {2010: 50},
 {2011: 50},
 {2012: 50},
 {2013: 50},
 {2014: 50},
 {2015: 50},
 {2016: 50},
 {2017: 50},
 {2018: 50},
 {2019: 50}]

In [21]:
climate['AT_mean'].describe()

count    1.095750e+06
mean     1.620686e+01
std      1.124801e+01
min     -2.569911e+01
25%      7.790189e+00
50%      1.667775e+01
75%      2.532557e+01
max      4.852972e+01
Name: AT_mean, dtype: float64

In [22]:
climate['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019])

In [23]:
climate = climate.sort_values(['city_name', 'year', 'month'])
climate

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
920430,1960,1,1960-01-01,36001,Albany,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658
920431,1960,1,1960-01-02,36001,Albany,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692
920432,1960,1,1960-01-03,36001,Albany,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212
920433,1960,1,1960-01-04,36001,Albany,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169
920434,1960,1,1960-01-05,36001,Albany,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508
...,...,...,...,...,...,...,...,...,...,...,...
328720,2019,12,2019-12-27,40143,Tulsa,9.324806,3.188396,15.461216,9.856630,3.321969,16.781241
328721,2019,12,2019-12-28,40143,Tulsa,10.919987,6.090799,15.749176,11.614218,6.371573,17.117941
328722,2019,12,2019-12-29,40143,Tulsa,9.419642,4.413086,14.426197,9.960367,4.600517,15.580466
328723,2019,12,2019-12-30,40143,Tulsa,4.814748,0.652293,8.977202,5.022408,0.709106,9.477184


### find temperature data during 1981 to 2010

In [24]:
year_temp = [year for year in range(1981, 2011)]

In [25]:
temp = climate[climate['year'].isin(year_temp)]
temp

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
928101,1981,1,1981-01-01,36001,Albany,-13.010622,-17.088109,-8.933135,-12.773773,-16.666624,-8.832458
928102,1981,1,1981-01-02,36001,Albany,-10.055733,-14.977587,-5.133880,-9.923142,-14.656974,-5.104323
928103,1981,1,1981-01-03,36001,Albany,-13.646982,-20.944347,-6.349617,-13.384197,-20.313775,-6.303968
928104,1981,1,1981-01-04,36001,Albany,-20.129528,-27.156104,-13.102953,-19.545513,-26.137207,-12.862413
928105,1981,1,1981-01-05,36001,Albany,-18.218619,-23.987399,-12.449840,-17.738922,-23.173324,-12.234875
...,...,...,...,...,...,...,...,...,...,...,...
325433,2010,12,2010-12-27,40143,Tulsa,-1.708393,-8.535238,5.118452,-1.685310,-8.444760,5.342278
325434,2010,12,2010-12-28,40143,Tulsa,1.920808,-4.446112,8.287729,2.010414,-4.422628,8.728146
325435,2010,12,2010-12-29,40143,Tulsa,7.184651,2.119199,12.250103,7.539349,2.214927,13.101360
325436,2010,12,2010-12-30,40143,Tulsa,14.128174,9.294666,18.961681,15.237380,9.823680,20.957473


In [26]:
temp['mean_temp'].describe()

count    547850.000000
mean         14.552994
std           9.678522
min         -26.686908
25%           7.596616
50%          15.398938
75%          22.481225
max          38.176219
Name: mean_temp, dtype: float64

In [27]:
city_numbers = []
for year in year_temp:
    dict_year = {}
    df_year = temp[temp['year'] == year]
    numbers = len(df_year['city_name'].unique())
    dict_year[year] = numbers
    city_numbers.append(dict_year)

In [28]:
city_numbers

[{1981: 50},
 {1982: 50},
 {1983: 50},
 {1984: 50},
 {1985: 50},
 {1986: 50},
 {1987: 50},
 {1988: 50},
 {1989: 50},
 {1990: 50},
 {1991: 50},
 {1992: 50},
 {1993: 50},
 {1994: 50},
 {1995: 50},
 {1996: 50},
 {1997: 50},
 {1998: 50},
 {1999: 50},
 {2000: 50},
 {2001: 50},
 {2002: 50},
 {2003: 50},
 {2004: 50},
 {2005: 50},
 {2006: 50},
 {2007: 50},
 {2008: 50},
 {2009: 50},
 {2010: 50}]

In [29]:
import datetime

temp['date'] = pd.to_datetime(temp['date'])
temp.head()

/tmp/ipykernel_1349533/108427414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['date'] = pd.to_datetime(temp['date'])


,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
928101,1981,1,1981-01-01,36001,Albany,-13.010622,-17.088109,-8.933135,-12.773773,-16.666624,-8.832458
928102,1981,1,1981-01-02,36001,Albany,-10.055733,-14.977587,-5.133880,-9.923142,-14.656974,-5.104323
928103,1981,1,1981-01-03,36001,Albany,-13.646982,-20.944347,-6.349617,-13.384197,-20.313775,-6.303968
928104,1981,1,1981-01-04,36001,Albany,-20.129528,-27.156104,-13.102953,-19.545513,-26.137207,-12.862413
928105,1981,1,1981-01-05,36001,Albany,-18.218619,-23.987399,-12.449840,-17.738922,-23.173324,-12.234875


In [30]:
temp_valid = temp[(temp['month'] == 7) | (temp['month'] == 8)]
temp_valid

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
928282,1981,7,1981-07-01,36001,Albany,21.471490,13.813391,29.129589,24.074127,14.876260,34.363752
928283,1981,7,1981-07-02,36001,Albany,21.955519,17.752517,26.158522,24.688166,19.493703,30.216889
928284,1981,7,1981-07-03,36001,Albany,22.154882,16.649029,27.660734,24.942357,18.177728,32.286800
928285,1981,7,1981-07-04,36001,Albany,21.862821,17.744575,25.981068,24.570229,19.484164,29.975833
928286,1981,7,1981-07-05,36001,Albany,21.323068,17.491559,25.154576,23.886709,19.180822,28.862394
...,...,...,...,...,...,...,...,...,...,...,...
325311,2010,8,2010-08-27,40143,Tulsa,22.743002,13.694677,31.791328,25.696660,14.740402,38.272178
325312,2010,8,2010-08-28,40143,Tulsa,24.518575,15.593415,33.443736,28.015741,16.935671,40.800317
325313,2010,8,2010-08-29,40143,Tulsa,27.085653,20.183794,33.987512,31.488105,22.461366,41.650508
325314,2010,8,2010-08-30,40143,Tulsa,28.404342,22.384344,34.424341,33.331494,25.235863,42.340263


In [31]:
temp_valid['mean_temp'].isna().sum()

0

In [32]:
temp_valid['max_temp'].isna().sum()

0

In [33]:
temp_valid['min_temp'].isna().sum()

0

In [34]:
min_AT_p85 = temp_valid.groupby(['city_name'])['AT_min'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
min_AT_p85.columns = ['city_name', 'AT_min_p85']
min_AT_p85

,city_name,AT_min_p85
0,Albany,19.868846
1,Albuquerque,16.668383
2,Allentown,21.260238
3,Atlanta,24.511237
4,Austin,27.036225
5,Baltimore,25.779365
6,Baton Rouge,26.646748
7,Birmingham,24.826357
8,Boston,22.765927
9,Buffalo,20.463265


In [35]:
climate

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
920430,1960,1,1960-01-01,36001,Albany,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658
920431,1960,1,1960-01-02,36001,Albany,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692
920432,1960,1,1960-01-03,36001,Albany,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212
920433,1960,1,1960-01-04,36001,Albany,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169
920434,1960,1,1960-01-05,36001,Albany,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508
...,...,...,...,...,...,...,...,...,...,...,...
328720,2019,12,2019-12-27,40143,Tulsa,9.324806,3.188396,15.461216,9.856630,3.321969,16.781241
328721,2019,12,2019-12-28,40143,Tulsa,10.919987,6.090799,15.749176,11.614218,6.371573,17.117941
328722,2019,12,2019-12-29,40143,Tulsa,9.419642,4.413086,14.426197,9.960367,4.600517,15.580466
328723,2019,12,2019-12-30,40143,Tulsa,4.814748,0.652293,8.977202,5.022408,0.709106,9.477184


### Find the heatwave Count

In [36]:
def findList(city_name, AT_min_p85):
    temp_p85_list = []
    temp_p85_list.append(city_name)
    temp_p85_list.append(AT_min_p85)
    return temp_p85_list

In [37]:
min_AT_p85['temp_p85_list'] = min_AT_p85.apply(lambda x: findList(x['city_name'], x['AT_min_p85']), axis=1)
min_AT_p85.head()

,city_name,AT_min_p85,temp_p85_list
0,Albany,19.868846,"[Albany, 19.868845773249163]"
1,Albuquerque,16.668383,"[Albuquerque, 16.668382957268324]"
2,Allentown,21.260238,"[Allentown, 21.260238020346623]"
3,Atlanta,24.511237,"[Atlanta, 24.51123653430137]"
4,Austin,27.036225,"[Austin, 27.036224664464918]"


In [38]:
temp_p85_list = min_AT_p85['temp_p85_list'].tolist()
temp_p85_list[:5]

[['Albany', 19.868845773249163],
 ['Albuquerque', 16.668382957268324],
 ['Allentown', 21.260238020346623],
 ['Atlanta', 24.51123653430137],
 ['Austin', 27.036224664464918]]

In [39]:
def findHeatwaveCount(temp, city, temp_p85):
    df = temp[temp['city_name'] == city]
    c1 = df['AT_min'].ge(temp_p85)
    N = 2
    g = (c1 != c1.shift()).cumsum()
    df1 = df.assign(
        cnt=df.groupby(g).date.transform('count')
      , n=df.groupby(g).agg('cumcount')
      , g=g
    )
    c2 = df1.cnt.ge(N)
    c3 = df1.n.mod(N).eq(0) & df1.n.le(df1.cnt-N)
    df['flag'] = np.where(c1 & c2 & c3, 1, 0)
    del(df1)
    return df

In [40]:
climate

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
920430,1960,1,1960-01-01,36001,Albany,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658
920431,1960,1,1960-01-02,36001,Albany,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692
920432,1960,1,1960-01-03,36001,Albany,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212
920433,1960,1,1960-01-04,36001,Albany,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169
920434,1960,1,1960-01-05,36001,Albany,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508
...,...,...,...,...,...,...,...,...,...,...,...
328720,2019,12,2019-12-27,40143,Tulsa,9.324806,3.188396,15.461216,9.856630,3.321969,16.781241
328721,2019,12,2019-12-28,40143,Tulsa,10.919987,6.090799,15.749176,11.614218,6.371573,17.117941
328722,2019,12,2019-12-29,40143,Tulsa,9.419642,4.413086,14.426197,9.960367,4.600517,15.580466
328723,2019,12,2019-12-30,40143,Tulsa,4.814748,0.652293,8.977202,5.022408,0.709106,9.477184


In [41]:
df = climate.merge(min_AT_p85, on='city_name', how='left')

In [42]:
df.head()

,year,month,date,fips,city_name,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max,AT_min_p85,temp_p85_list
0,1960,1,1960-01-01,36001,Albany,-7.164961,-13.460060,-0.869862,-7.104883,-13.205012,-0.838658,19.868846,"[Albany, 19.868845773249163]"
1,1960,1,1960-01-02,36001,Albany,-9.151526,-17.303679,-0.999372,-9.044999,-16.871308,-0.969692,19.868846,"[Albany, 19.868845773249163]"
2,1960,1,1960-01-03,36001,Albany,-0.437482,-6.560136,5.685172,-0.400462,-6.511033,5.941212,19.868846,"[Albany, 19.868845773249163]"
3,1960,1,1960-01-04,36001,Albany,0.865364,-2.200579,3.931308,0.926935,-2.180386,4.096169,19.868846,"[Albany, 19.868845773249163]"
4,1960,1,1960-01-05,36001,Albany,-3.921627,-8.507685,0.664432,-3.901230,-8.417892,0.721508,19.868846,"[Albany, 19.868845773249163]"


In [43]:
# Function to count heatwaves
def count_heatwaves(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Count the groups that are at least two days long
    heatwave_count = (group_lengths >= 2).sum()

    return heatwave_count

In [44]:
# Apply the function to each year and fips
heatwave_counts = df.groupby(['year', 'month', 'city_name']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))

# Reset the index
heatwave_counts = heatwave_counts.reset_index(name='heatwave_count')
heatwave_counts

,year,month,city_name,heatwave_count
0,1960,1,Albany,0
1,1960,1,Albuquerque,0
2,1960,1,Allentown,0
3,1960,1,Atlanta,0
4,1960,1,Austin,0
...,...,...,...,...
35995,2019,12,Seattle,0
35996,2019,12,St. Louis,0
35997,2019,12,Tampa,0
35998,2019,12,Tucson,0


In [46]:
heatwave_counts['heatwave_count'].describe()

count    36093.000000
mean         1.544482
std          1.544958
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max         11.000000
Name: heatwave_count, dtype: float64

In [46]:
heatwave_counts.to_csv('city_heatwave/PRISM_city_heatwave/city_heatwave_count_monthly_1960_2019.csv')

In [47]:
heatwave_counts['heatwave_count'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [48]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and has columns 'city', 'date', 'temperature'
# Assuming percentile_dict is your dictionary with 85th percentile temperatures for each city

# Sort by city and date
df = df.sort_values(['city_name', 'month', 'date'])

In [49]:
years = [year for year in range(1960, 2020)]

In [50]:
# Define a function to calculate heatwave durations for each city
def calculate_heatwave(df, years):
    pattern = []
    for year in years:
        # Create a boolean series where True indicates a day is part of a heatwave
        city_df = df[df['year'] == year] 
        for city in city_df['city_name'].unique():
            df_city = city_df[city_df['city_name'] == city]
            percentile = df_city['AT_min_p85']
            is_heatwave = df_city['AT_min'] > percentile

            # Identify where True values change to False and vice versa,
            # and assign each group of consecutive True values a unique identifier
            heatwave_id = (is_heatwave.diff() != 0).cumsum()

            # Group by the identifiers and count the size of each group
            heatwave_lengths = heatwave_id[is_heatwave].value_counts()

            # Filter out heatwaves that are less than 2 days long
            heatwave_lengths = heatwave_lengths[heatwave_lengths >= 2]

            # Create a DataFrame with the start date of each heatwave and its duration
            heatwaves = pd.DataFrame({
                'city_name': city,
                'duration': heatwave_lengths.values,
                'year': year
            })
            pattern.append(heatwaves)
    
    return pattern

In [51]:
pattern = calculate_heatwave(df, years)
heatwave_durations = pd.concat(pattern)
heatwave_durations

,city_name,duration,year
0,Albuquerque,2,1960
0,Atlanta,2,1960
0,Austin,3,1960
0,Baton Rouge,2,1960
1,Baton Rouge,2,1960
...,...,...,...
1,Tucson,5,2019
2,Tucson,5,2019
3,Tucson,3,2019
0,Tulsa,3,2019


In [52]:
# Function to calculate heatwave durations
def calculate_heatwave_durations(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Filter for groups that are at least two days long, and return their lengths
    heatwave_durations = group_lengths[group_lengths >= 2]

    return heatwave_durations

In [53]:
# Apply the function to each year and fips
heatwave_durations = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_durations(group['AT_min'], group['AT_min_p85'].iloc[0]))
# This will result in a series where each element is a series of heat wave durations
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_durations = heatwave_durations.explode().reset_index(name='heatwave_duration')
heatwave_durations

,year,city_name,AT_min,heatwave_duration
0,1960,Albuquerque,8,2
1,1960,Atlanta,2,2
2,1960,Austin,2,3
3,1960,Baton Rouge,6,2
4,1960,Baton Rouge,8,2
...,...,...,...,...
7282,2019,Tucson,6,5
7283,2019,Tucson,10,5
7284,2019,Tucson,4,3
7285,2019,Tulsa,2,3


In [54]:
heatwave_durations.to_csv('city_heatwave/PRISM_city_heatwave/city_heatwave_duration_1960_2019.csv')

In [55]:
df['date'] = pd.to_datetime(df['date'])

In [56]:
# Function to calculate heatwave dates
def calculate_heatwave_dates(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()
    
    # Get the dates of each group
    group_dates = series.loc[is_above_threshold, 'date'].groupby(heatwave_groups)

    # Filter for groups that are at least two days long, and return their dates
    heatwave_dates = group_dates.filter(lambda dates: len(dates) >= 2)

    return heatwave_dates

In [57]:
# Apply the function to each year and fips
heatwave_dates = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_dates(group[['date', 'AT_min']], group['AT_min_p85'].iloc[0]))
heatwave_dates

year  city_name           
1960  Albuquerque  22125     1960-07-29
                   22126     1960-07-30
      Atlanta      65927     1960-07-01
                   65928     1960-07-02
      Austin       87894     1960-08-22
                                ...    
2019  Tulsa        1095583   2019-07-18
                   1095584   2019-07-19
                   1095585   2019-07-20
                   1095607   2019-08-11
                   1095608   2019-08-12
Name: date, Length: 25033, dtype: datetime64[ns]

In [58]:
# The season length for each year and fips is the difference between the max and min date
season_lengths = heatwave_dates.groupby(level=['year','city_name']).apply(lambda dates: dates.max() - dates.min())
season_lengths = season_lengths.explode().reset_index(name='heatwave_season_length')
season_lengths

,year,city_name,heatwave_season_length
0,1960,Albuquerque,1 days
1,1960,Atlanta,1 days
2,1960,Austin,2 days
3,1960,Baton Rouge,5 days
4,1960,Chicago,45 days
...,...,...,...
2454,2019,Seattle,87 days
2455,2019,St. Louis,20 days
2456,2019,Tampa,85 days
2457,2019,Tucson,27 days


In [59]:
def findDaysNumber(days):
    days = str(days)
    return int(days.split()[0])+1

In [60]:
season_lengths['heatwave_length'] = season_lengths['heatwave_season_length'].apply(findDaysNumber)
season_lengths

,year,city_name,heatwave_season_length,heatwave_length
0,1960,Albuquerque,1 days,2
1,1960,Atlanta,1 days,2
2,1960,Austin,2 days,3
3,1960,Baton Rouge,5 days,6
4,1960,Chicago,45 days,46
...,...,...,...,...
2454,2019,Seattle,87 days,88
2455,2019,St. Louis,20 days,21
2456,2019,Tampa,85 days,86
2457,2019,Tucson,27 days,28


In [61]:
season_lengths = season_lengths.drop('heatwave_season_length', axis = 1)
season_lengths = season_lengths.rename(columns = {'heatwave_length': 'heatwave_season_length'})
season_lengths

,year,city_name,heatwave_season_length
0,1960,Albuquerque,2
1,1960,Atlanta,2
2,1960,Austin,3
3,1960,Baton Rouge,6
4,1960,Chicago,46
...,...,...,...
2454,2019,Seattle,88
2455,2019,St. Louis,21
2456,2019,Tampa,86
2457,2019,Tucson,28


In [62]:
season_lengths.to_csv('city_heatwave/PRISM_city_heatwave/city_heatwave_season_length_1960_2019.csv')

In [63]:
def celsius_to_fahrenheit(c):
    f = c * 9/5 + 32
    return f

In [64]:
df['AT_min'] = df['AT_min'].apply(celsius_to_fahrenheit)
df['AT_min_p85'] = df['AT_min_p85'].apply(celsius_to_fahrenheit)

In [65]:
# Function to calculate average exceeded temperatures above threshold during heatwaves
def calculate_heatwave_exceeded_temp_avgs(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Get the mean of exceeded temperatures (temperature - threshold) above threshold in each group
    group_exceeded_temp_avgs = (series.loc[is_above_threshold, 'AT_min'] - threshold).groupby(heatwave_groups).mean()

    # Filter for groups that are at least two days long, and return their exceeded temperature averages
    heatwave_exceeded_temp_avgs = group_exceeded_temp_avgs[group_exceeded_temp_avgs.index.map(is_above_threshold.groupby(heatwave_groups).size() >= 2)]

    return heatwave_exceeded_temp_avgs

In [66]:
# Apply the function to each year and fips
heatwave_exceeded_temp_avgs = df.groupby(['year', 'city_name']).apply(lambda group: calculate_heatwave_exceeded_temp_avgs(group[['AT_min']], group['AT_min_p85'].iloc[0]))
heatwave_exceeded_temp_avgs 

year  city_name    AT_min
1960  Albuquerque  8         3.875958
      Atlanta      2         1.390346
      Austin       2         0.599297
      Baton Rouge  6         1.084929
                   8         0.582137
                               ...   
2019  Tucson       6         1.693777
                   10        2.035266
                   12        1.243578
      Tulsa        2         1.811961
                   4         0.345105
Name: AT_min, Length: 7287, dtype: float64

In [67]:
# This will result in a series where each element is a series of heat wave exceeded temperature averages
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.explode().reset_index(name='heatwave_exceeded_temp_avg')
heatwave_exceeded_temp_avgs

,year,city_name,AT_min,heatwave_exceeded_temp_avg
0,1960,Albuquerque,8,3.875958
1,1960,Atlanta,2,1.390346
2,1960,Austin,2,0.599297
3,1960,Baton Rouge,6,1.084929
4,1960,Baton Rouge,8,0.582137
...,...,...,...,...
7282,2019,Tucson,6,1.693777
7283,2019,Tucson,10,2.035266
7284,2019,Tucson,12,1.243578
7285,2019,Tulsa,2,1.811961


In [68]:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.drop('AT_min', axis = 1)
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.rename(columns = {'heatwave_exceeded_temp_avg': 'heatwave_intensity'})
heatwave_exceeded_temp_avgs

,year,city_name,heatwave_intensity
0,1960,Albuquerque,3.875958
1,1960,Atlanta,1.390346
2,1960,Austin,0.599297
3,1960,Baton Rouge,1.084929
4,1960,Baton Rouge,0.582137
...,...,...,...
7282,2019,Tucson,1.693777
7283,2019,Tucson,2.035266
7284,2019,Tucson,1.243578
7285,2019,Tulsa,1.811961


In [69]:
heatwave_exceeded_temp_avgs.to_csv('city_heatwave/PRISM_city_heatwave/city_heatwave_intensity_1960_2019.csv')